<a href="https://colab.research.google.com/github/UroojAshfa/Mini-ML-projects/blob/main/smartms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from huggingface_hub import login

login(token="your_token")

In [10]:
#importing libraries
import os
import PyPDF2
import spacy
import streamlit as st
import nltk
from transformers import pipeline
from huggingface_hub import hf_hub_download
import whisper
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import re
from textblob import TextBlob
from spacy import displacy

# Initialize spaCy model and other NLP tools
nlp = spacy.load("en_core_web_sm")

# Download the Gemma 2 model
summarizer = pipeline("summarization", model="google/gemma-2-2b-it")


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
The model 'Gemma2ForCausalLM' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditi

In [11]:
#Text Preprocessing: This function will clean the text by removing stop words, special characters, and tokenizing the text.


def preprocess_text(text):
    # Tokenization and text cleaning
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    cleaned_text = " ".join(tokens)
    return cleaned_text

In [12]:
 #Text Summarization Using Gemma 2

def summarize_text(text):
    summary = summarizer(text, max_length=200, min_length=50, do_sample=False)
    return summary[0]['summary_text']

In [13]:
#Sentiment Analysis Using Gemma 2: For sentiment analysis, we will assume Gemma 2 has the required model. If not, we can use TextBlob or other models from Hugging Face.


def sentiment_analysis(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

In [14]:
#Keyword Extraction Using TF-IDF

def extract_keywords(text):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = np.array(vectorizer.get_feature_names_out())
    sorted_idx = tfidf_matrix.sum(axis=0).argsort()[::-1]
    top_keywords = feature_names[sorted_idx[:10]]
    return top_keywords

In [15]:
#Decision/Action Item Extraction: We’ll extract action items using dependency parsing or simple keyword matching.


def extract_decisions(text):
    doc = nlp(text)
    decisions = []
    for sent in doc.sents:
        for token in sent:
            if token.dep_ == "ROOT" and token.pos_ == "VERB":
                decisions.append(sent.text)
    return decisions

In [17]:
#backend
def handle_file_upload(uploaded_file):
    if uploaded_file is not None:
        # Extract text from the PDF
        pdf_reader = PyPDF2.PdfReader(uploaded_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

        # Preprocess text
        cleaned_text = preprocess_text(text)

        # Summarize text
        summary = summarize_text(cleaned_text)

        # Sentiment analysis
        sentiment = sentiment_analysis(text)

        # Extract Keywords
        keywords = extract_keywords(text)

        # Extract decisions/action items
        decisions = extract_decisions(text)

        return {
            'summary': summary,
            'sentiment': sentiment,
            'keywords': keywords,
            'decisions': decisions
        }
    else:
        return None


In [18]:
#Build the Streamlit UI for File Upload and Result Display

def main():
    # Streamlit Layout
    st.title("Smart Meeting Summarizer")
    st.markdown("Upload your meeting notes or PDF file below:")

    # File upload section
    uploaded_file = st.file_uploader("Choose a PDF file", type=["pdf"])

    if uploaded_file:
        # Process the uploaded file
        results = handle_file_upload(uploaded_file)

        if results:
            st.subheader("Summary")
            st.write(results['summary'])

            st.subheader("Sentiment Analysis")
            st.write(results['sentiment'])

            st.subheader("Keywords")
            st.write(", ".join(results['keywords']))

            st.subheader("Decisions/Action Items")
            st.write("\n".join(results['decisions']))

            # Option to download results
            download_option = st.selectbox("Download results as", ["JSON", "Markdown"])

            if download_option == "JSON":
                json_data = json.dumps(results, indent=4)
                st.download_button(label="Download JSON", data=json_data, file_name="summary.json")

            elif download_option == "Markdown":
                markdown_data = f"# Meeting Summary\n\n## Summary\n{results['summary']}\n\n## Sentiment\n{results['sentiment']}\n\n## Keywords\n" + ", ".join(results['keywords']) + "\n\n## Decisions/Action Items\n" + "\n".join(results['decisions'])
                st.download_button(label="Download Markdown", data=markdown_data, file_name="summary.md")
        else:
            st.write("No file uploaded. Please upload a PDF file.")


In [19]:
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [23]:
#Write Streamlit Code to a Python Script

%%writefile app.py
import streamlit as st

def main():
    st.title("Hello from Streamlit")
    st.write("This is a simple Streamlit app running from Google Colab!")

if __name__ == "__main__":
    main()

Overwriting app.py


In [25]:
from pyngrok import ngrok
import os

# Set up the Streamlit app port using pyngrok
public_url = ngrok.connect(8501)

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Print the public URL for the app
print(f"Streamlit app is live at: {public_url}")


ERROR:pyngrok.process.ngrok:t=2024-11-29T14:35:33+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-29T14:35:33+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-11-29T14:35:33+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.